In [1]:
%load_ext autoreload
%autoreload 2

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from hatedetection import HateSpeechAnalyzer
analyzer = HateSpeechAnalyzer.load_contextualized_model()

In [2]:

import ijson
from tqdm.auto import tqdm



path = "../../data/news-all-final.json"

num_articles = 537_200

data = []

num_comments = 0

LIMIT = 5e3
with open(path) as f:
    articles = ijson.items(f, 'item')

    for i, article in tqdm(enumerate(articles), total=num_articles):
        num_comments += len(article["comments"])

        context = article["title"] if "title" in article else article["text"]

        for comment in article["comments"]:
            data.append({
                "text": comment["text"],
                "context": context,
                "id": comment["tweet_id"],
            })
        
        if i > LIMIT:
            break

        

8.5M de comentarios. Ok, es un montonazo

In [3]:
import pandas as pd
from pandarallel import pandarallel
from hatedetection.preprocessing import preprocess_tweet

pandarallel.initialize()

df = pd.DataFrame(data)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
device = "cuda"

model = analyzer.base_model
model = model.to(device)
model.eval()
tokenizer = analyzer.tokenizer

In [5]:
df["text"] = df["text"].parallel_apply(preprocess_tweet)
df["context"] = df["context"].parallel_apply(preprocess_tweet)

In [6]:
from hatedetection.training import tokenize
from torch.utils.data import DataLoader
from datasets import Dataset, Value, ClassLabel, Features

features = Features({
    'id': Value('uint64'),
    'context': Value('string'),
    'text': Value('string'),
})

batch_size = 24

dataset = Dataset.from_pandas(df, features=features)

dataset = dataset.map(
    lambda x: tokenizer(x["text"], x["context"], padding="longest", truncation='longest_first'), 
    batched=True, batch_size=batch_size, 
)



In [7]:
def format_dataset(dataset):
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])
    return dataset

dataset = format_dataset(dataset)

dataloader = DataLoader(dataset, batch_size=batch_size)

In [8]:

from hatedetection import extended_hate_categories

outputs = []

for batch in tqdm(dataloader):
    batch = {k:v.to(device) for k, v in batch.items() if k != "id"}
    outs = model(**batch)
    outputs.append((outs.logits > 0).cpu().numpy())


In [10]:
import numpy as np
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 300

df[extended_hate_categories] = np.vstack(outputs)
df["HATEFUL"] = df[extended_hate_categories[1:]].sum(axis=1) > 0

df[extended_hate_categories].sum()


CALLS         205
WOMEN         679
LGBTI         237
RACISM        411
CLASS         217
POLITICS      898
DISABLED      352
APPEARANCE    907
CRIMINAL      238
dtype: int64

In [13]:
pd.options.display = 200
pd.options.display.max_colwidth = 300


df[df["LGBTI"]][["text", "context"] + extended_hate_categories]

,text,context,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
194,[USER] TROLA LESBIANA,El poderoso mensaje que escondió Natalie Portman en su vestido de los Oscars 2020,False,True,True,False,False,False,False,False,False
207,"[USER] El fútbol se vuelve más ""no seas trolo Man"" cada día. Una pelota que viene tan rápida no te da tiempo a sacar la mano, es obvio que no hay intención. Las nuevas reglas para cobrar mano están hechas para los llorones y organizadores q quieren q los partidos salgan 8 a 5","El detalle en el gol de Racing: la ""mano de VAR"" de Cvitanich en la jugada previa que debió sancionar el árbitro",False,False,True,False,False,False,False,False,False
219,"[USER] Rojo puto, jugadores a todos la concha de su madre. [USER]",Las cuatro rojas en el clásico de Avellaneda: ¿Loustau acertó en todas las expulsiones?,False,False,True,True,False,False,False,False,False
591,[USER] Vestida de payasa.,"Billie Eilish, la antidiva de la alfombra roja de los Oscar, con uñas a lo Rosalía y traje grafitero",False,True,True,False,False,False,False,False,False
968,"[USER] Degenerado no, mal gusto","Carmen Barbieri: ""Un degenerado me tocó la cola a la salida del teatro""",False,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
58929,[USER] Bueno. La verdad qué. No te apoyo. Eso de qué. La noche sera diversa o No sera habla de una intolerancia qué es impropia del movimiento LGTB.,Intendencia citó a los dueños de local denunciado por discriminación a mujer trans,False,False,True,False,False,False,False,False,False
58950,[USER] Pijasos en el orto se merecen nomás estos maricones asesinos,"Té con leche, truco y biblia: ¿Cómo pasan los días en la cárcel los rugbiers?",True,False,True,False,False,False,False,False,True
59109,"[USER] Degenerado, animal, bestia humana. Escrachenlo",Tiene 12 años y logró grabar cómo un familiar la abusaba: está prófugo,False,False,True,False,False,False,False,False,False
59208,"[USER] Agarrenlo... Degenerado, perverso, bestia y animal.","Vive en un country de Moreno y es gerente de una empresa: quién es Leandro Martínez, el tío abusador filmado ""in fraganti""",False,False,True,False,False,False,False,False,False


In [10]:
import torch

